In [ ]:


from __future__ import annotations
import datetime as dt
import json, math, os, warnings
from pathlib import Path
from typing import Dict, List, Optional, Tuple
import copy


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


import torch
print(f"CUDA available in PyTorch: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print("GPU is accessible to PyTorch despite NVML warning")
    print(f"GPU Device: {torch.cuda.get_device_name(0)}")


import torch.nn as nn
import torch.optim as optim



from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import t


pm = None; ETSModel = None; Holt = None; SimpleExpSmoothing = None
ConvergenceWarning = Warning; ValueWarning = Warning

try:
    from statsmodels.tsa.exponential_smoothing.ets import ETSModel
    from statsmodels.tsa.api import Holt, SimpleExpSmoothing
    from statsmodels.tools.sm_exceptions import ConvergenceWarning, ValueWarning
except ImportError:
    print("Warning: statsmodels not fully installed or outdated.")

try:
    import pmdarima as pm
except (ImportError, ValueError) as e:
    print(f"\nWarning: Failed to import pmdarima. Auto-ARIMA baseline will be skipped.")
    pm = None

mrmr = None
try:
    import mrmr
except ImportError:
    print("Warning: mrmr-selection not installed. Multivariate mode will fail if used.")

try:
    import optuna
    from optuna.samplers import TPESampler
    from optuna.pruners import SuccessiveHalvingPruner
except ImportError:
    print("Warning: Optuna not installed. The main execution block will be skipped.")
    optuna = None


ROOT = Path("yourpath")

BASE_OUT = GDRIVE_ROOT / "36_forecast_outputs_GRU_rec_1stepopt"
BASE_OUT.mkdir(parents=True, exist_ok=True)
STORAGE = f"sqlite:///{BASE_OUT / 'lstm_rec1stepopt_optuna_study.db'}"
MODEL_TYPE = "GRU_Rec1StepOpt"


FORECAST_HORIZON = 36; SEASONAL_PERIOD = 12


OPTUNA_N_TRIALS = 50; MAX_EPOCHS_OPTUNA = 30
PATIENCE_ES = 5; N_CV_SPLITS = 5

VALIDATION_SIZE_HPO = 36


MAX_EPOCHS_FINAL = 150; WEIGHT_DECAY = 1e-4; RANDOM_STATE = 42


MC_SAMPLES_FOR_CI = 100; EPSILON = 1e-8


if torch: torch.manual_seed(RANDOM_STATE)
np.random.seed(RANDOM_STATE)
device = torch.device("cuda" if torch and torch.cuda.is_available() else "cpu")
print("Using device:", device)



try:
    if torch:
        torch.backends.cudnn.benchmark = True
        if torch.cuda.is_available():
            if hasattr(torch.backends.cuda.matmul, 'allow_tf32'):
                    torch.backends.cuda.matmul.allow_tf32 = True
            if hasattr(torch, 'set_float32_matmul_precision'):
                torch.set_float32_matmul_precision("medium")
except Exception:
    pass

FUTURE_EXOG_POLICY = "persist"


if nn:
    class GRUForecaster(nn.Module):
        """
        GRU model adapted for 1 step ahead forecasting.
        """
        def __init__(self, input_size: int, hidden_size: int,
                     output_size: int, dropout: float, num_layers: int):
            super().__init__()


            self.rnn = nn.GRU(
                input_size,
                hidden_size,
                num_layers=num_layers,
                batch_first=True,
                dropout=dropout if num_layers > 1 else 0.0,
            )
            self.dropout = nn.Dropout(dropout)

            self.fc = nn.Linear(hidden_size, output_size)

        def forward(self, x):

            out, _ = self.rnn(x)

            out = self.dropout(out[:, -1, :])

            return self.fc(out)

def build_model(hp: Dict, n_vars: int) -> nn.Module:
    if not nn: return None
    mdl = GRUForecaster(
        input_size=n_vars,
        hidden_size=int(hp["hidden_size"]),
        output_size=1,
        dropout=float(hp["dropout_rate"]),
        num_layers=int(hp["num_layers"]),
    ).to(device)
    try:

        if hasattr(torch, 'compile'):
             return torch.compile(mdl, mode="reduce-overhead")
        return mdl
    except Exception:
        return mdl

def smape_loss_np(y_true, y_pred, eps: float = EPSILON) -> float:
    y_true = np.asarray(y_true); y_pred = np.asarray(y_pred)
    num = np.abs(y_true - y_pred)
    den = (np.abs(y_true) + np.abs(y_pred) + eps) / 2.0
    return np.mean(num / den)

def calculate_metrics(y_true: np.ndarray, y_pred: np.ndarray, insample: Optional[np.ndarray] = None):
    if y_true.ndim == 1 or (y_true.ndim > 1 and y_true.shape[1] == 1):
        y_true = y_true.ravel(); y_pred = y_pred.ravel()

    smape = smape_loss_np(y_true, y_pred)
    mase = rmsse = np.nan

    if insample is not None and len(insample) > SEASONAL_PERIOD:
        d = np.mean(np.abs(np.diff(insample, n=SEASONAL_PERIOD)))
        mae = np.mean(np.abs(y_true - y_pred)); mse = np.mean((y_true - y_pred) ** 2)
        mase = mae / (d + EPSILON)
        denom_rmsse = np.mean((np.diff(insample, n=SEASONAL_PERIOD))**2)
        rmsse = np.sqrt(mse / (denom_rmsse + EPSILON))

    return {"SMAPE": smape, "MASE": mase, "RMSSE": rmsse}

def diebold_mariano_test(actuals: np.ndarray, pred1: np.ndarray, pred2: np.ndarray, horizon: int):

    actuals = actuals.ravel(); pred1 = pred1.ravel(); pred2 = pred2.ravel()
    if len(actuals) != len(pred1) or len(actuals) != len(pred2): return np.nan, np.nan
    N = len(actuals)
    if N == 0: return np.nan, np.nan

    loss1 = (actuals - pred1)**2; loss2 = (actuals - pred2)**2
    d = loss1 - loss2; d_mean = np.mean(d)

    q = horizon; gamma = np.zeros(q + 1)
    for k in range(q + 1):
        if k == 0: gamma[k] = np.var(d, ddof=0)
        else:
            try:
                cov = np.cov(d[k:], d[:N-k], ddof=0)
                if cov.ndim == 2: gamma[k] = cov[0, 1]
                else: gamma[k] = 0
            except Exception: gamma[k] = 0

    V = gamma[0] + 2 * np.sum([(1 - k/(q+1)) * gamma[k] for k in range(1, q+1)])

    if V <= 1e-9:
        if abs(d_mean) < 1e-9: return 0, 1.0
        else: return np.inf * np.sign(d_mean), 0.0

    DM_stat = d_mean / np.sqrt(V / N)
    p_value = 2 * (1 - t.cdf(np.abs(DM_stat), df=N-1))
    return DM_stat, p_value


def fit_smoother(series, method, alpha=None, beta=None):
    if method == "NS" or SimpleExpSmoothing is None or Holt is None: return None
    try:
        if method == "DES":
            return Holt(series, initialization_method="estimated").fit(smoothing_level=alpha, smoothing_trend=beta, optimized=False)
        if method == "ES":
             return SimpleExpSmoothing(series, initialization_method="estimated").fit(smoothing_level=alpha, optimized=False)
    except Exception: return None
    return None

def apply_fitted_smoother(fitter, df: pd.DataFrame, target: str) -> pd.DataFrame:
    if fitter is None: return df
    n = len(df); fitted_vals = fitter.fittedvalues
    if len(fitted_vals) == 0: return df

    if len(fitted_vals) == n: values = fitted_vals
    elif len(fitted_vals) < n: values = pd.concat([df[target].iloc[:n-len(fitted_vals)], fitted_vals])
    else: values = fitted_vals[:n]

    out = df.copy()
    if len(values) == n: out[target] = values.values
    else: return df

    out[target].bfill(inplace=True)
    return out


def create_sequences_1step(df_in: pd.DataFrame, target: str,
                           features: List[str], num_lags: int) -> Tuple[np.ndarray, np.ndarray, List[str]]:
    """
    Creates input sequences (X) and target values (Y) for 1-step ahead forecasting.

    X: (N_samples, num_lags, N_features)
    Y: (N_samples, 1)
    """
    cols = [target] + features
    data = df_in[cols].values
    N = data.shape[0]

    X, Y = [], []


    for i in range(N - num_lags):

        x_seq = data[i : i + num_lags]

        y_val = data[i + num_lags, 0]

        X.append(x_seq)
        Y.append(y_val)

    return np.array(X), np.array(Y).reshape(-1, 1), cols


def select_mrmr_features(df_window: pd.DataFrame, target: str, k: int, num_lags: int) -> List[str]:

    if k == 0 or mrmr is None: return []
    lagged = []; potential_features = [c for c in df_window.columns if c != target]
    if not potential_features: return []

    for col in [target] + potential_features:
        for l in range(1, num_lags + 1):
            lagged.append(df_window[col].shift(l).rename(f"{col}_lag_{l}"))

    df_lagged = pd.concat([df_window[[target]], *lagged], axis=1).dropna()
    if df_lagged.empty: return []

    X_cols = [c for c in df_lagged.columns if "_lag_" in c and not c.startswith(f"{target}_lag_")]
    if not X_cols: return []

    X = df_lagged[X_cols].ffill().fillna(0); y = df_lagged[target].values.ravel()
    K_eff = min(k, len(X_cols))

    try:
        selected_lagged = mrmr.mrmr_regression(X=X, y=y, K=K_eff)
        selected_base = list({s.split("_lag_")[0] for s in selected_lagged})
        return selected_base
    except Exception:
        return []

def mc_pred(model: nn.Module, inp: torch.Tensor, mc_samples: int):
    """Generates Monte Carlo predictions if dropout is active. Ensures output is (mc_samples, 1)."""
    if mc_samples == 1:
        model.eval()
        with torch.no_grad():
            preds = model(inp).cpu().numpy()

        preds = np.asarray(preds)
        if preds.ndim == 1:
            preds = preds.reshape(-1, 1)
        elif preds.ndim > 2:
            preds = preds.reshape(preds.shape[0], -1)[:, :1]
        return preds

    model.train()
    with torch.no_grad():
        inp_mc = inp.repeat(mc_samples, 1, 1)
        preds = model(inp_mc).cpu().numpy()
    preds = np.asarray(preds)

    if preds.ndim == 1:
        preds = preds.reshape(-1, 1)
    elif preds.ndim > 2:
        preds = preds.reshape(preds.shape[0], -1)[:, :1]
    elif preds.shape[1] != 1:
        preds = preds[:, :1]
    return preds




def make_objective(tgt: str, train_val_df: pd.DataFrame,
                   k: int, mode: str):


    tscv = TimeSeriesSplit(n_splits=N_CV_SPLITS, test_size=VALIDATION_SIZE_HPO)

    def objective(trial: optuna.Trial):

        hp = {
            "hidden_size": trial.suggest_categorical("hidden_size", [64, 128, 256, 512]),
            "num_layers": trial.suggest_int("num_layers", 1, 3),
            "dropout_rate": trial.suggest_float("dropout_rate", 0.1, 0.5, step=0.1),
            "lr": trial.suggest_categorical("lr", [1e-4, 5e-4, 1e-3]),
            "batch_size": trial.suggest_categorical("batch_size", [32, 64, 128]),
            "n_lags": trial.suggest_int("n_lags", SEASONAL_PERIOD, SEASONAL_PERIOD * 4, step=SEASONAL_PERIOD),
            "smoothing_method": trial.suggest_categorical("smoothing_method", ["NS", "ES", "DES"]),
            "smoothing_alpha": trial.suggest_float("smoothing_alpha", 0.1, 0.6),
            "smoothing_beta": trial.suggest_float("smoothing_beta", 0.05, 0.6),
        }

        n_lags = hp["n_lags"]
        smapes_cv = []


        for tr_idx, val_idx in tscv.split(train_val_df):
            if len(tr_idx) < n_lags + 1:
                continue

            df_tr = train_val_df.iloc[tr_idx]


            val_start_idx = val_idx[0] - n_lags
            if val_start_idx < 0:
                 continue

            df_va_context = train_val_df.iloc[val_start_idx : val_idx[-1] + 1]



            smoother = fit_smoother(df_tr[tgt], hp["smoothing_method"], hp["smoothing_alpha"], hp["smoothing_beta"])
            df_tr_sm = apply_fitted_smoother(smoother, df_tr, tgt)


            df_va_context_sm = apply_fitted_smoother(smoother, df_va_context, tgt)



            if mode == "MULTI":
                base_feats = select_mrmr_features(df_tr_sm, tgt, k, num_lags=SEASONAL_PERIOD)
            else:
                base_feats = []

            n_vars = len([tgt, *base_feats])


            X_tr, Y_tr, _ = create_sequences_1step(df_tr_sm, tgt, base_feats, n_lags)

            X_va, Y_va, _ = create_sequences_1step(df_va_context_sm, tgt, base_feats, n_lags)


            if X_tr.shape[0] == 0 or X_va.shape[0] == 0:
                continue


            X_tr_reshaped = X_tr.reshape(-1, n_vars)
            sc_X = MinMaxScaler().fit(X_tr_reshaped); sc_y = MinMaxScaler().fit(Y_tr)

            X_tr_scaled = sc_X.transform(X_tr.reshape(-1, n_vars)).reshape(X_tr.shape)
            Y_tr_scaled = sc_y.transform(Y_tr)
            X_va_scaled = sc_X.transform(X_va.reshape(-1, n_vars)).reshape(X_va.shape)



            Xtr_t = torch.tensor(X_tr_scaled, dtype=torch.float32)
            Ytr_t = torch.tensor(Y_tr_scaled, dtype=torch.float32)
            Xva_t = torch.tensor(X_va_scaled, dtype=torch.float32).to(device)

            dl = torch.utils.data.DataLoader(
                torch.utils.data.TensorDataset(Xtr_t, Ytr_t),
                batch_size=hp["batch_size"], shuffle=True)


            mdl = build_model(hp, n_vars)
            crit = nn.L1Loss()
            opt = optim.AdamW(mdl.parameters(), lr=hp["lr"], weight_decay=WEIGHT_DECAY)

            best_fold_smape = float("inf")
            no_imp = 0

            for ep in range(MAX_EPOCHS_OPTUNA):
                mdl.train()
                for xb, yb in dl:
                    xb, yb = xb.to(device), yb.to(device)
                    opt.zero_grad(set_to_none=True)
                    with torch.autocast(device.type, enabled=(device.type == "cuda")):
                        preds = mdl(xb)

                        loss = crit(preds, yb)

                    if torch.isnan(loss): return float("inf")

                    loss.backward()
                    torch.nn.utils.clip_grad_norm_(mdl.parameters(), 1.0)
                    opt.step()


                mdl.eval()
                with torch.no_grad():
                    preds_scaled = mdl(Xva_t).cpu().numpy()

                    preds_inv = sc_y.inverse_transform(preds_scaled)


                    s = smape_loss_np(Y_va, preds_inv)


                trial.report(s, ep)
                if trial.should_prune():
                    raise optuna.TrialPruned()

                if s < best_fold_smape - 1e-4:
                    best_fold_smape = s; no_imp = 0
                else:
                    no_imp += 1

                if no_imp >= PATIENCE_ES:
                    break

            smapes_cv.append(best_fold_smape)

        if not smapes_cv:
            return float("inf")


        return float(np.mean(smapes_cv))

    return objective



def train_final_model(hp: Dict, df_train: pd.DataFrame, tgt: str, base_feats: List[str], use_early_stopping: bool = True):
    """
    Trains the 1-step LSTM model with optimized hyperparameters.
    """
    n_lags = hp["n_lags"]
    n_vars = len([tgt, *base_feats])


    smoother = fit_smoother(df_train[tgt], hp.get("smoothing_method", "NS"), hp.get("smoothing_alpha"), hp.get("smoothing_beta"))
    df_train_sm = apply_fitted_smoother(smoother, df_train, tgt)


    X, Y, _ = create_sequences_1step(df_train_sm, tgt, base_feats, n_lags)
    if X.shape[0] == 0: return None, None, None


    if use_early_stopping and X.shape[0] > 20:
         split_idx = int(X.shape[0] * 0.8)
         X_tr, Y_tr = X[:split_idx], Y[:split_idx]
         X_va, Y_va = X[split_idx:], Y[split_idx:]
    else:
        X_tr, Y_tr = X, Y
        X_va, Y_va = None, None


    X_tr_reshaped = X_tr.reshape(-1, n_vars)
    sc_X = MinMaxScaler().fit(X_tr_reshaped); sc_y = MinMaxScaler().fit(Y_tr)

    X_tr_scaled = sc_X.transform(X_tr.reshape(-1, n_vars)).reshape(X_tr.shape)
    Y_tr_scaled = sc_y.transform(Y_tr)

    Xtr_t = torch.tensor(X_tr_scaled, dtype=torch.float32)
    Ytr_t = torch.tensor(Y_tr_scaled, dtype=torch.float32)

    if X_va is not None:
        X_va_scaled = sc_X.transform(X_va.reshape(-1, n_vars)).reshape(X_va.shape)
        Y_va_scaled = sc_y.transform(Y_va)
        Xva_t = torch.tensor(X_va_scaled, dtype=torch.float32).to(device)
        Yva_t = torch.tensor(Y_va_scaled, dtype=torch.float32).to(device)
    else:
        Xva_t, Yva_t = None, None

    dl = torch.utils.data.DataLoader(
        torch.utils.data.TensorDataset(Xtr_t, Ytr_t),
        batch_size=hp["batch_size"], shuffle=True)


    mdl = build_model(hp, n_vars)
    if mdl is None: return None, None, None

    crit = nn.L1Loss()
    opt = optim.AdamW(mdl.parameters(), lr=hp["lr"], weight_decay=WEIGHT_DECAY)


    best_val_loss = float("inf"); no_imp = 0; best_model_state = None

    if Xva_t is None:
         best_model_state = copy.deepcopy(mdl.state_dict())

    for ep in range(MAX_EPOCHS_FINAL):
        mdl.train()
        for xb, yb in dl:
            xb, yb = xb.to(device), yb.to(device)
            opt.zero_grad(set_to_none=True)
            with torch.autocast(device.type, enabled=(device.type == "cuda")):
                preds = mdl(xb)
                loss = crit(preds, yb)

            if torch.isnan(loss): return None, None, None

            loss.backward()
            torch.nn.utils.clip_grad_norm_(mdl.parameters(), 1.0)
            opt.step()


        if Xva_t is not None:
            mdl.eval()
            with torch.no_grad():
                 with torch.autocast(device.type, enabled=(device.type == "cuda")):
                    val_preds = mdl(Xva_t)
                    val_loss = crit(val_preds, Yva_t).item()

            if val_loss < best_val_loss - 1e-4:
                best_val_loss = val_loss; no_imp = 0
                best_model_state = copy.deepcopy(mdl.state_dict())
            else:
                no_imp += 1

            if no_imp >= PATIENCE_ES: break

    if best_model_state:
         mdl.load_state_dict(best_model_state)

    return mdl, sc_X, sc_y


def run_hpo_pipeline(df_dev: pd.DataFrame, targets: List[str], k: int, mode: str) -> Dict:
    best_cfgs = {}
    print(f"\n--- Starting HPO Pipeline (Model: {MODEL_TYPE}, Mode: {mode}) ---")

    for tgt in targets:
        print(f"\nOptimizing Target: {tgt}")

        study_name = f"{MODEL_TYPE}_{mode}_{tgt}_HPO"
        try:
            study = optuna.create_study(
                study_name=study_name,
                direction="minimize",
                sampler=TPESampler(seed=RANDOM_STATE),
                pruner=SuccessiveHalvingPruner(min_resource=4, reduction_factor=3),
                storage=STORAGE,
                load_if_exists=True
            )
        except Exception as e:
            print(f"Could not create/load study: {e}")
            continue

        if len(study.trials) < OPTUNA_N_TRIALS:
            objective_fn = make_objective(tgt, df_dev, k, mode)
            try:
                study.optimize(
                    objective_fn,
                    n_trials=OPTUNA_N_TRIALS - len(study.trials),
                    n_jobs=1,
                    show_progress_bar=True
                )
            except Exception as e:
                print(f"Optimization failed: {e}")
                continue

        if study.best_trial is None:
            print(f"HPO failed for {tgt}. Skipping.")
            continue

        print(f"Best CV 1-Step SMAPE for {tgt}: {study.best_value:.4f}")
        best_hp = study.best_params


        if mode == "MULTI":
            smoother = fit_smoother(
                df_dev[tgt],
                best_hp.get("smoothing_method", "NS"),
                best_hp.get("smoothing_alpha", 0.5),
                best_hp.get("smoothing_beta", 0.1)
            )
            df_dev_sm = apply_fitted_smoother(smoother, df_dev, tgt)
            final_feats = select_mrmr_features(df_dev_sm, tgt, k, num_lags=SEASONAL_PERIOD)
        else:
            final_feats = []

        best_cfgs[tgt] = {
            "smape_dev_1step": study.best_value,
            "hyperparams": best_hp,
            "base_features": final_feats,
            "n_vars_per_step": len([tgt, *final_feats]),
            "n_lags": best_hp["n_lags"]
        }

    return best_cfgs



def forecast_arima(series: pd.Series, horizon: int):
    if pm is None: return np.full(horizon, np.nan)
    try:
        model = pm.auto_arima(series, start_p=1, start_q=1, max_p=5, max_q=5, m=SEASONAL_PERIOD,
                              start_P=0, seasonal=True, d=None, D=1, trace=False,
                              error_action='ignore', suppress_warnings=True, stepwise=True)
        forecast = model.predict(n_periods=horizon)
        return np.asarray(forecast)
    except Exception: return np.full(horizon, np.nan)

def forecast_ets(series: pd.Series, horizon: int):
    if ETSModel is None: return np.full(horizon, np.nan)
    try:
        model = ETSModel(series, error="add", trend="add", seasonal="add",
                         damped_trend=True, seasonal_periods=SEASONAL_PERIOD)
        fit = model.fit(disp=False, optimized=True)
        forecast = fit.forecast(horizon)
        return np.asarray(forecast)
    except Exception:

        return np.repeat(series.iloc[-1], horizon)

def forecast_exogenous_variables(df_hist: pd.DataFrame, features: List[str], horizon: int) -> Dict[str, np.ndarray]:
    """
    Rigorously forecasts future values of exogenous variables using ETS.
    """
    forecasts = {}
    for feat in features:

        forecasts[feat] = forecast_ets(df_hist[feat], horizon)
    return forecasts

def _append_next_point(current_input_t: torch.Tensor,
                       pred_next_scaled: torch.Tensor,
                       n_vars: int,
                       policy: str = FUTURE_EXOG_POLICY) -> torch.Tensor:
    """
    Build the (Batch, 1, n_vars) point to append after predicting the next step,
    WITHOUT using any *future* exogenous values.

    Inputs are in *scaled* space:
      - current_input_t: (B, L, n_vars)
      - pred_next_scaled: (B, 1) predicted target (scaled)
    """
    B = current_input_t.size(0)
    new_point = pred_next_scaled.unsqueeze(1)

    if n_vars == 1:
        return new_point

    if policy == "persist":

        exog_last = current_input_t[:, -1:, 1:]
        return torch.cat([new_point, exog_last], dim=2)

    if policy == "none":
        pad = torch.zeros((B, 1, n_vars - 1),
                          device=current_input_t.device,
                          dtype=current_input_t.dtype)
        return torch.cat([new_point, pad], dim=2)

    if policy == "forecast":
        raise RuntimeError("FUTURE_EXOG_POLICY='forecast' not allowed in Track A. Use 'persist' or 'none'.")


def generate_forecast_recursive(
    df_hist: pd.DataFrame, target: str, cfg: Dict, horizon: int,
    mc_samples: int):
    """
    Trains the 1-step model and generates a H-step ahead forecast recursively.
    """
    if not cfg or not cfg.get("hyperparams"): return None

    hp = cfg["hyperparams"]; feats = cfg["base_features"]
    n_lags = cfg["n_lags"]; n_vars = cfg["n_vars_per_step"]


    mdl, sc_X, sc_y = train_final_model(hp, df_hist, target, feats, use_early_stopping=True)

    if mdl is None: return None


    if FUTURE_EXOG_POLICY == "forecast" and feats:
        exog_forecasts = forecast_exogenous_variables(df_hist, feats, horizon)
    else:
        exog_forecasts = None



    smoother = fit_smoother(df_hist[target], hp.get("smoothing_method", "NS"), hp.get("smoothing_alpha"), hp.get("smoothing_beta"))
    df_hist_sm = apply_fitted_smoother(smoother, df_hist, target)


    input_data = df_hist_sm[[target] + feats].iloc[-n_lags:].values


    input_scaled = sc_X.transform(input_data.reshape(-1, n_vars)).reshape(1, n_lags, n_vars)
    current_input_t = torch.tensor(input_scaled, dtype=torch.float32, device=device)


    use_mc = mc_samples > 1 and hp.get("dropout_rate", 0) > 0
    samples = mc_samples if use_mc else 1


    all_forecasts = np.zeros((samples, horizon))

    for h in range(horizon):

        preds_mc_scaled = mc_pred(mdl, current_input_t, samples)

        preds_mc_scaled = np.asarray(preds_mc_scaled)
        if preds_mc_scaled.ndim == 2:

            preds_mc_scaled = preds_mc_scaled.reshape(preds_mc_scaled.shape[0], -1)[:, 0]
        else:
            preds_mc_scaled = preds_mc_scaled.reshape(-1)




        all_forecasts[:, h] = preds_mc_scaled.ravel()


        if h < horizon - 1:

          if current_input_t.shape[0] == 1 and samples > 1:
              current_input_t = current_input_t.repeat(samples, 1, 1)

          if exog_forecasts is None:

              pred_scaled_t = torch.tensor(preds_mc_scaled, dtype=torch.float32, device=device).unsqueeze(1)
              new_point_t = _append_next_point(current_input_t, pred_scaled_t, n_vars, FUTURE_EXOG_POLICY)
          else:

              new_point = preds_mc_scaled.reshape(samples, 1)


              exog_vals_h = np.array([exog_forecasts[f][h] for f in feats]).reshape(1, -1)
              dummy_for_scaling = np.hstack([np.zeros((1, 1)), exog_vals_h])
              scaled_dummy = sc_X.transform(dummy_for_scaling)
              scaled_exog_h = scaled_dummy[:, 1:].reshape(1, len(feats))
              scaled_exog_repeated = np.repeat(scaled_exog_h, samples, axis=0)

              new_point_np = np.hstack([new_point, scaled_exog_repeated])
              new_point_t = torch.tensor(new_point_np.reshape(samples, 1, n_vars),
                                        dtype=torch.float32, device=device)


          current_input_t = torch.cat([current_input_t[:, 1:, :], new_point_t], dim=1)



    preds_inv = sc_y.inverse_transform(all_forecasts.reshape(-1, 1)).reshape(samples, horizon)


    if use_mc:
        forecasts = np.median(preds_inv, axis=0)
        lowers = np.percentile(preds_inv, 2.5, axis=0)
        uppers = np.percentile(preds_inv, 97.5, axis=0)
    else:
        forecasts = preds_inv[0]
        lowers = np.full(horizon, np.nan); uppers = np.full(horizon, np.nan)


    forecasts[forecasts < 0] = 0
    if use_mc:
        lowers[lowers < 0] = 0; uppers[uppers < 0] = 0

    return pd.DataFrame({"Forecast": forecasts, "Lower_CI": lowers, "Upper_CI": uppers})




def plot_error_horizon_analysis(results_df: pd.DataFrame, model_type: str, output_dir: Path):
    """Analyzes and plots the SMAPE error across the 36-month horizon."""


    model_results = results_df[results_df['Model'].str.startswith(model_type)]

    if model_results.empty:
        print("No data for error horizon analysis.")
        return


    ape_cols = [f'APE_H{h+1}' for h in range(FORECAST_HORIZON)]

    if ape_cols[0] not in model_results.columns:
        print("APE columns not found in results dataframe. Cannot perform analysis.")
        return


    melted_df = pd.melt(model_results, id_vars=['Target', 'Model'], value_vars=ape_cols, var_name='Horizon', value_name='APE')


    melted_df['Horizon_Step'] = melted_df['Horizon'].str.replace('APE_H', '').astype(int)


    melted_df['SMAPE'] = melted_df['APE'] * 100


    plt.figure(figsize=(12, 6))
    sns.lineplot(data=melted_df, x='Horizon_Step', y='SMAPE', hue='Model', errorbar='sd')
    plt.title(f'Error Horizon Analysis ({model_type} Strategies)')
    plt.xlabel('Forecast Horizon (Months)')
    plt.ylabel('Average SMAPE (%)')
    plt.xticks(range(1, FORECAST_HORIZON + 1, 2))
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.legend(title='Strategy')

    plt.tight_layout()
    plot_path = output_dir / "error_horizon_analysis.png"
    plt.savefig(plot_path)
    print(f"\nError Horizon Analysis plot saved to: {plot_path}")
    plt.show()



if __name__ == "__main__":
    if not torch or not optuna:
        print("\nExecution aborted due to missing critical dependencies (PyTorch or Optuna).")



    if ConvergenceWarning: warnings.simplefilter("ignore", ConvergenceWarning)
    if ValueWarning: warnings.simplefilter("ignore", ValueWarning)
    warnings.simplefilter("ignore", UserWarning)

    if optuna:
        try: optuna.logging.set_verbosity(optuna.logging.WARNING)
        except Exception: pass

    --
    DF_PATH = ROOT / "yourdatahere"


    if DF_PATH.exists():
        df_raw = pd.read_excel(DF_PATH)
    else:
         print("Data file not found. Proceeding with MOCK DATA.")


    date_col = next((c for c in df_raw.columns
                     if c.lower().strip() == "date"), None)
    if date_col:
        df_raw[date_col] = pd.to_datetime(df_raw[date_col])
        df_raw.sort_values(date_col, inplace=True)
        df_raw.set_index(date_col, inplace=True)

    df_raw.columns = df_raw.columns.str.lower().str.strip()


    for col in df_raw.columns:
        if df_raw[col].dtype == "object":
             try:
                df_raw[col] = pd.to_numeric(
                    df_raw[col].str.replace(r"[^\d.\-]", "", regex=True),
                    errors="coerce")
             except AttributeError:
                 df_raw[col] = pd.to_numeric(df_raw[col], errors="coerce")
    df_raw = df_raw.ffill().fillna(0)


    df_dev = df_raw.iloc[:-FORECAST_HORIZON].copy()
    df_test = df_raw.iloc[-FORECAST_HORIZON:].copy()


    targets = [col for col in df_raw.columns if 'journal article' in col]
    if not targets:
            raise RuntimeError("No valid targets found in data.")


    K_FEATURES = 10
    best_cfgs_modes = {}


    for MODE in ["UNI", "MULTI"]:
        if MODE == "MULTI" and (mrmr is None or len(df_raw.columns) < 2):
            print(f"Skipping {MODE} mode."); continue


        best_cfgs = run_hpo_pipeline(df_dev, targets, k=K_FEATURES, mode=MODE)
        best_cfgs_modes[MODE] = best_cfgs


        best_path = BASE_OUT / f"best_hyperparameters_{MODE}.json"
        try:

            tmp = best_path.with_suffix(".tmp")
            with open(tmp, 'w') as f:
                json.dump(best_cfgs, f, indent=2)
            tmp.replace(best_path)
        except Exception as e:
            print(f"Could not save HPO results for {MODE}: {e}")

    print("\n--- Starting Test Set (Hold-out) Evaluation ---")
    test_results = []; baseline_cache = {}


    evaluation_records = []

    for MODE in best_cfgs_modes.keys():
        best_cfgs = best_cfgs_modes[MODE]
        MODEL_MODE_NAME = f"{MODEL_TYPE}_{MODE}"

        for tgt in targets:
            cfg = best_cfgs.get(tgt)
            if not cfg: continue



            fc_df = generate_forecast_recursive(df_dev, tgt, cfg, FORECAST_HORIZON, mc_samples=1)

            if fc_df is None:
                print(f"Skipping {tgt} ({MODE}) due to model training failure."); continue

            y_true = df_test[tgt].values.ravel()
            y_pred_model = fc_df["Forecast"].values.ravel()
            insample_data = df_dev[tgt].values

            model_m = calculate_metrics(y_true, y_pred_model, insample_data)


            ape = np.abs(y_true - y_pred_model) / ((np.abs(y_true) + np.abs(y_pred_model) + EPSILON) / 2.0)
            ape_dict = {f'APE_H{h+1}': ape[h] for h in range(FORECAST_HORIZON)}


            record = {
                "Target": tgt, "Model": MODEL_MODE_NAME,
                **{f"{k}": v for k, v in model_m.items()},
                **ape_dict
            }


            if tgt not in baseline_cache:
                print(f"Calculating Baselines for {tgt}...")

                arima_pred = forecast_arima(df_dev[tgt], FORECAST_HORIZON)
                ets_pred = forecast_ets(df_dev[tgt], FORECAST_HORIZON)
                baseline_cache[tgt] = {"ARIMA": arima_pred, "ETS": ets_pred}


            for name, pred in baseline_cache[tgt].items():
                 if np.isnan(pred).all():
                     metrics = {"SMAPE": np.nan, "MASE": np.nan, "RMSSE": np.nan}
                     ape_dict_base = {f'APE_H{h+1}': np.nan for h in range(FORECAST_HORIZON)}
                 else:
                    metrics = calculate_metrics(y_true, pred, insample_data)
                    ape_base = np.abs(y_true - pred) / ((np.abs(y_true) + np.abs(pred) + EPSILON) / 2.0)
                    ape_dict_base = {f'APE_H{h+1}': ape_base[h] for h in range(FORECAST_HORIZON)}


                 baseline_record = {
                     "Target": tgt, "Model": name,
                     **metrics,
                     **ape_dict_base
                 }
                 evaluation_records.append(baseline_record)


            arima_smape = evaluation_records[-2]['SMAPE'] if len(evaluation_records) >= 2 else np.inf
            ets_smape = evaluation_records[-1]['SMAPE'] if len(evaluation_records) >= 1 else np.inf

            if np.isnan(arima_smape): arima_smape = np.inf
            if np.isnan(ets_smape): ets_smape = np.inf

            best_stat_name = None
            if arima_smape != np.inf or ets_smape != np.inf:
                if arima_smape <= ets_smape:
                    best_stat_name = "ARIMA"; best_stat_pred = baseline_cache[tgt]["ARIMA"]
                else:
                    best_stat_name = "ETS"; best_stat_pred = baseline_cache[tgt]["ETS"]

            if best_stat_name:
                if np.isnan(y_pred_model).any() or np.isnan(best_stat_pred).any():
                    dm_stat, dm_p = np.nan, np.nan
                else:
                    dm_stat, dm_p = diebold_mariano_test(y_true, y_pred_model, best_stat_pred, FORECAST_HORIZON)

                record[f"DM_vs_{best_stat_name}_Stat"] = dm_stat
                record[f"DM_vs_{best_stat_name}_Pval"] = dm_p

            evaluation_records.append(record)


    df_results = pd.DataFrame(evaluation_records)

    if not df_results.empty:

        df_results.to_csv(BASE_OUT / "test_evaluation_detailed.csv", index=False)


        try:
            summary_metrics = ['SMAPE', 'MASE', 'RMSSE']

            dm_cols = [col for col in df_results.columns if col.startswith('DM_')]

            pivot_df = df_results.pivot_table(index='Target', columns='Model', values=summary_metrics + dm_cols)


            pivot_df.columns = ['_'.join(col).strip() for col in pivot_df.columns.values]
            pivot_df = pivot_df.reset_index()
            pivot_df.to_csv(BASE_OUT / "test_evaluation_summary.csv", index=False)
            print("\nTest Evaluation Summary saved.")

        except Exception as e:
            print(f"Error creating summary pivot table: {e}")



        print("\n--- Performing Error Horizon Analysis ---")
        plot_error_horizon_analysis(df_results, MODEL_TYPE, BASE_OUT)



    print("\n--- Generating Final 36-Month Future Forecasts (using full data) ---")
    df_full = pd.concat([df_dev, df_test])

    for MODE, best_cfgs in best_cfgs_modes.items():
        for tgt in targets:
            cfg = best_cfgs.get(tgt)
            if not cfg: continue


            fc_df = generate_forecast_recursive(df_full, tgt, cfg, FORECAST_HORIZON, MC_SAMPLES_FOR_CI)

            if fc_df is None:
                print(f"Skipping future projection for {tgt} ({MODE})."); continue


            safe_name = "".join(c if c.isalnum() else "_" for c in tgt)
            fc_df.to_csv(BASE_OUT / f"forecast_future_36m_{MODE.lower()}_{safe_name}.csv", index=False)

    print(f"\nAll tasks finished successfully. Outputs are located at: {BASE_OUT}")